# Audit Planning

## Initial Imports

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import os
import yaml
from crewai import Agent, Task, Crew
from crewai.llm import LLM

## Loading Tasks and Agents YAML files

In [ ]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yml',
    'tasks': 'config/tasks.yml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [ ]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool, JSONSearchTool, TXTSearchTool

In [ ]:
pcaob_guidlines_tool = JSONSearchTool(config={
        "llm": {
            "provider": "vertexai",
            "config": {
                "model": "gemini-2.0-flash-lite-001",
            },
        },
        "embedder": {
            "provider": "vertexai",
            "config": {
                "model": "text-embedding-004",
            },
        },
    },json_path='./data/compliance.json')

website_search_tool = WebsiteSearchTool(config={
        "llm": {
            "provider": "vertexai",
            "config": {
                "model": "gemini-2.0-flash-lite-001",
            },
        },
        "embedder": {
            "provider": "vertexai",
            "config": {
                "model": "text-embedding-004",
            },
        },
    })
auditpulse_file_tool = TXTSearchTool(config={
        "llm": {
            "provider": "vertexai",
            "config": {
                "model": "gemini-2.0-flash-lite-001",
            },
        },
        "embedder": {
            "provider": "vertexai",
            "config": {
                "model": "text-embedding-004",
            },
        },
    },
    file_path="./data/AuditPulseInfo.md")

## Setup Multi LLM models

> Note: The 'llama-3.1-70b-versatil' model was deprecated. As a result, we updated it to the 'llama-3.3-70b-versatil' model in January 2025.

In [ ]:
llm = LLM(
    model="vertex_ai/gemini-2.0-flash-lite-001",
    max_tokens=	4096,
    context_window_size=950000,
)

## Creating Crew, Agents, and Tasks

In [ ]:
phase = 'audit_planning'
# Initialize Audit Planning Agent
audit_planning_agent = Agent(
    config=agents_config['audit_planning_agent'],
    tools=[
        SerperDevTool(), 
        ScrapeWebsiteTool(), 
        website_search_tool, 
        pcaob_guidlines_tool,
        auditpulse_file_tool
    ],
    llm=llm,
    respect_context_window=True,
    max_rpm=10,
    cache=True,
    max_retry_limit=10
)

# Creating CrewAI Tasks

preliminary_engagement_task = Task(
    config=tasks_config['audit_planning']['preliminary_engagement_review'],
    async_execution=False,
    agent=audit_planning_agent,
    output_file=f'./output/{phase}/preliminary_engagement_task.md',
)

business_risk_task = Task(
    config=tasks_config['audit_planning']['business_risk_and_fraud_assessment'],
    async_execution=False,
    agent=audit_planning_agent,
    output_file=f'./output/{phase}/business_risk_task.md',
)

internal_control_task = Task(
    config=tasks_config['audit_planning']['internal_control_system_evaluation'],
    async_execution=False,
    agent=audit_planning_agent,
    context=[preliminary_engagement_task, business_risk_task],
    output_file=f'./output/{phase}/internal_control_task.md',
)

audit_strategy_task = Task(
    config=tasks_config['audit_planning']['audit_strategy_and_team_allocation'],
    async_execution=False,
    agent=audit_planning_agent,
    context=[internal_control_task],
    output_file=f'./output/{phase}/audit_strategy_task.md',
)

# Creating the CrewAI Workflow

audit_planning_crew = Crew(
    agents=[audit_planning_agent],
    tasks=[
        preliminary_engagement_task,
        business_risk_task,
        internal_control_task,
        audit_strategy_task
    ],
    verbose=True,
    output_log_file="./logs/audit_planning.txt"
)

## Kicking off the Crew

In [ ]:
result = audit_planning_crew.kickoff(inputs={
                'audit_firm':'AuditPulse',
                'company_name': 'Apple Inc.',
                'central_index_key': 320193,
                'company_ticker': 'AAPL',
                'year': '2024'
            },
        )

In [ ]:
import glob
phase = 'audit_planning'
base_path = f'./output/{phase}'
tasks_files = ['preliminary_engagement_task.md','business_risk_task.md','internal_control_task.md','audit_strategy_task.md']
with open(os.path.join(base_path,'audit_planning_report.md'),'w') as final_report_file:
    for task_file in tasks_files:
        file = os.path.join(base_path,task_file)
        with open(file,'r') as task_report_file:
            final_report_file.write(task_report_file.read().lstrip('```markdown').lstrip('```').rstrip('```'))
            final_report_file.write(f'\n')

## Report

In [ ]:
from IPython.display import display, Markdown
display(Markdown(f'./output/{phase}/audit_planning_report.md'))

## Cost

In [ ]:
usage_metrics = audit_planning_crew.usage_metrics

cost_per_million_prompt = 0.075
cost_per_million_completion = 0.30

prompt_cost = (usage_metrics.prompt_tokens / 1_000_000) * cost_per_million_prompt
completion_cost = (usage_metrics.completion_tokens / 1_000_000) * cost_per_million_completion

total_cost = prompt_cost + completion_cost
print(f'Total Execution Cost: ${total_cost}')

## Testing

In [ ]:
audit_planning_crew.test(n_iterations=3, 
                         eval_llm=llm,
                         inputs={
                                'audit_firm':'AuditPulse',
                                'company_name': 'Apple Inc.',
                                'central_index_key': 320193,
                                'company_ticker': 'AAPL',
                                'year': '2024'
                            }
                         )

## Training

In [ ]:
audit_planning_crew.train(n_iterations=1, filename='audit_planning_crew_training.pkl')
# audit_planning_crew.load('audit_planning_crew_training.pkl')